In [1]:
import pandas as pd
import numpy as np
import glob
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import os
from sklearn.neighbors import NearestNeighbors

## Loading the data

In [2]:
def load_data(data_dir):
    """
    Load all the data from the data directory
    """
    data = []
    for file in glob.glob(os.path.join(data_dir, '*.csv')):
        data.append(pd.read_csv(file))
    return pd.concat(data, ignore_index=True)

In [5]:
path_to_train = r'../data/train/treino_parte1.csv'

In [6]:
train_df = pd.read_csv(path_to_train)
train_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [7]:
# Definir caminhos para os arquivos de treino (supondo que estejam nomeados como treino_parte_*.csv)
path_to_items = r"../data/items"

In [8]:
items_df = load_data(path_to_items)
items_df.head()

,page,url,issued,modified,title,body,caption
0,b2d67f99-e758-4719-adde-53926854ae38,http://g1.globo.com/sp/sao-paulo/eleicoes/2022...,2022-05-03 19:38:56+00:00,2022-05-03 20:54:49+00:00,Lula compara Arthur Lira a imperador do Japão,Ex-presidente Lula discursa em evento do Solid...,Petista voltou a criticar presidente da Câmara...
1,ddd9ba15-6026-4a9a-9204-08946b27be64,http://g1.globo.com/sp/sao-paulo/eleicoes/2022...,2022-05-04 16:54:14+00:00,2022-05-04 16:54:15+00:00,Voto facultativo terá peso inédito na eleição ...,Detalhe da urna eletrônica e a tecla confirma\...,Quantidade de eleitores de 16 a 17 anos e acim...
2,a06f8a15-b2ca-4363-b060-b821a158e35b,http://g1.globo.com/sp/sao-paulo/eleicoes/2022...,2022-05-23 19:18:56+00:00,2022-05-23 20:43:51+00:00,PT confirma conversa com economista Pérsio Ari...,"A presidente do PT, Gleisi Hoffmann, durante e...","Segundo Gleisi Hoffmann, Aloizio Mercadante te..."
3,55ab912a-2bac-46d9-9fcf-8e9be376f1b3,http://g1.globo.com/sp/sao-paulo/eleicoes/2022...,2022-08-11 13:12:31+00:00,2022-08-12 01:32:24+00:00,Ato pela democracia recorda mortos na ditadur...,Carta em defesa da democracia foi lida nesta q...,Evento reuniu milhares dentro e fora da Faculd...
4,6d7d0faf-58ce-43b7-bd22-b62a56e1fc41,http://g1.globo.com/politica/noticia/2022/01/2...,2022-01-28 21:32:08+00:00,2022-01-29 01:11:05+00:00,"Moro revela ter recebido R$ 3,6 milhões por 12...",Moro faz live para explicar pagamento por serv...,TCU apura possível 'conflito de interesse' na ...


In [9]:
# Lista de colunas que devem ser convertidas para listas
colunas_listas = [
    'history',
    'timestampHistory',
    'numberOfClicksHistory',
    'pageVisitsCountHistory',
    'timeOnPageHistory',
    'scrollPercentageHistory'
]

In [10]:
def converter_str_para_lista(df, colunas):
    for col in colunas:
        # Verifica se a coluna existe e se o valor não é nulo
        df[col] = df[col].apply(lambda x: x.split(", ") if pd.notnull(x) else [])
    return df

In [11]:
# Aplicar a conversão
train_df = converter_str_para_lista(train_df, colunas_listas)

In [12]:
# Exemplo: Verificando a coluna 'history' para um usuário específico
user_id = "2c1080975e257ed630e26679edbe4d5c850c65f3e09f655798b0bba9b42f2110"
print("Histórico do usuário:", train_df.loc[train_df['userId'] == user_id, 'history'].iloc[0])

Histórico do usuário: ['3325b5a1-979a-4cb3-82b6-63905c9edbe8', 'fe856057-f97d-419f-ab1c-97c5c3e0719c', 'd1da8d3f-bf30-4776-8d75-5f4e24cc134e', '8d7b7c79-4a54-4d5f-8d02-36805d60c498', '982578ac-ce37-4330-aec8-b44843e79f67', '083220a2-455c-4b64-b47a-807af2d6f00a', 'e752d7a2-d87b-4894-8c91-9743cc644b77', '96a30f4a-a9b2-44fd-89ae-4fd14e5cf246', '9a9b6a92-4a97-4c4d-99df-16fbe9546f0a', 'f03b8e40-56dd-410c-a04e-2711df683287', 'b409a984-c60d-4852-a55f-8e153ce9e69a', 'a1fb4a4d-4ad6-4dfe-9567-bceff5113d0b', '99d60eed-cda1-48fd-8349-00494ed9dd46', 'c511189f-5ced-4b0b-a06f-fcdb17264c24', 'c725a9d0-c658-4431-bbe9-5f592b56b990', '2d63c994-b72e-472b-ba73-c978916a6015', 'e1880fc6-c9dc-4565-b378-952f04e6a4d4', '370d60b4-7936-4239-9745-f1ecac5a1bbc', '5133418b-211f-4f58-9d25-86b46addfbc8', 'bbd29779-404b-4c34-8c4e-cddc0852d5b0', '21ac2572-76f3-4539-9369-bd7b629fd5dd', '179a4335-cc5b-4728-9a6c-a1d0a37a262c', '8bb97465-965f-4910-ac62-9d17af26b2bf', '5b902915-fab0-48a1-913f-3b12fe047372', 'ef5ec9cb-de05-4a

In [13]:
# Converter IDs para string, se necessário
train_df['userId'] = train_df['userId'].astype(str)

In [14]:
train_df.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"[c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039...","[1657146417045, 1657146605778, 1657146698738]","[76, 38, 41]","[20380, 21184, 35438]","[50.3, 18.18, 16.46]","[2, 1, 1]","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"[3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe85605...","[1656684240278, 1656761266729, 1656761528085, ...","[7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13,...","[6049, 210489, 8672, 10000, 30000, 123007, 996...","[25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, ...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"[04756569-593e-4133-a95a-83d35d43dbbd, 29b6b14...","[1656678946256, 1656701076495, 1656701882565, ...","[0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0...","[311274, 140000, 32515, 157018, 118689, 159243...","[67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71....","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"[1f2b9c2f-a2d2-4192-b009-09065da8ec23, 0475656...","[1658333312180, 1658404553818, 1658408449062, ...","[8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, ...","[182696, 91925, 30000, 273655, 126409, 42980, ...","[58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"[bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a...","[1658766608801, 1658766608801, 1660084035094, ...","[579, 579, 7, 2]","[801396, 801396, 10000, 10000]","[78.74, 78.74, 16.71, 9.34]","[7, 7, 1, 1]","1658766608801, 1658766608801, 1660084035094, 1..."


In [15]:
# Função para "explodir" as interações: gera uma linha para cada item no histórico do usuário
def explodir_interacoes(df):
    registros = []
    for idx, row in df.iterrows():
        user_id = row['userId']
        user_type = row['userType']
        history = row['history']
        timestamps = row['timestampHistory']
        clicks = row['numberOfClicksHistory']
        visits = row['pageVisitsCountHistory']
        
        # Para cada item no histórico, cria um registro único
        for i in range(len(history)):
            reg = {
                'userId': user_id,
                'userType': user_type,
                'itemId': history[i],
                'timestamp': timestamps[i],
                'clicks': clicks[i],
                'visits': visits[i]
            }
            registros.append(reg)
    df_interacoes = pd.DataFrame(registros)
    return df_interacoes

In [16]:
# Aplicar a função ao dataframe de treino (train_df já deve ter as colunas convertidas para listas)
df_interacoes = explodir_interacoes(train_df)
print("Exemplo de interações explosivas:")
print(df_interacoes.head())

Exemplo de interações explosivas:
                                              userId    userType  \
0  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
1  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
2  f98d1132f60d46883ce49583257104d15ce723b3bbda21...  Non-Logged   
3  2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...  Non-Logged   
4  2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...  Non-Logged   

                                 itemId      timestamp clicks visits  
0  c8aab885-433d-4e46-8066-479f40ba7fb2  1657146417045     76      2  
1  68d2039c-c9aa-456c-ac33-9b2e8677fba7  1657146605778     38      1  
2  13e423ce-1d69-4c78-bc18-e8c8f7271964  1657146698738     41      1  
3  3325b5a1-979a-4cb3-82b6-63905c9edbe8  1656684240278      7      1  
4  fe856057-f97d-419f-ab1c-97c5c3e0719c  1656761266729     80      1  


In [17]:
# Gerar os mapeamentos a partir do df_interacoes
unique_users = df_interacoes['userId'].unique()
user_id_to_index = {user: idx for idx, user in enumerate(unique_users)}

In [18]:
unique_items = df_interacoes['itemId'].unique()
item_id_to_index = {item: idx for idx, item in enumerate(unique_items)}

In [19]:
# Construir a matriz esparsa de interações
# Cada linha representa um usuário e cada coluna um item; o valor é a contagem de interações
rows = df_interacoes['userId'].map(user_id_to_index).values
cols = df_interacoes['itemId'].map(item_id_to_index).values
data = np.ones(len(df_interacoes))  # Peso 1 para cada interação

In [20]:
interaction_matrix = csr_matrix((data, (rows, cols)), shape=(len(unique_users), len(unique_items)))
print("Matriz de interação (esparsa):", interaction_matrix.shape)

Matriz de interação (esparsa): (100000, 108573)


## Collaborative filtering Model

In [21]:
# Definir o número de vizinhos para a busca (valor ajustável conforme a necessidade)
n_neighbors = 100

In [22]:
# Criar o modelo de vizinhos para o CF, utilizando distância cosseno (com "brute" para matrizes esparsas)
nn_model_cf = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_neighbors, n_jobs=-1)
nn_model_cf.fit(interaction_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=100)

In [23]:
def cf_recommend(user_id, top_n=10, n_neighbors=n_neighbors):
    """
    Recomenda itens para um usuário utilizando vizinhos mais próximos (CF via NearestNeighbors).
    Para um dado usuário, busca os n vizinhos mais próximos com base na interação,
    acumula os itens que eles consumiram (excluindo os já consumidos pelo usuário) e os ordena por similaridade agregada.
    """
    if user_id not in user_id_to_index:
        return []
    
    user_idx = user_id_to_index[user_id]
    
    # Buscar os vizinhos mais próximos para o usuário (a consulta retorna o próprio usuário com distância 0)
    distances, indices = nn_model_cf.kneighbors(interaction_matrix[user_idx], n_neighbors=n_neighbors)
    distances = distances.flatten()
    indices = indices.flatten()
    
    # Converter a distância em similaridade (1 - distância, pois a distância cosseno varia entre 0 e 1)
    # E acumular os itens dos vizinhos, ignorando os itens já consumidos pelo usuário
    user_items = set(interaction_matrix[user_idx].nonzero()[1])
    rec_scores = {}
    
    for neighbor_idx, dist in zip(indices, distances):
        # Ignorar o próprio usuário
        if neighbor_idx == user_idx:
            continue
        similarity = 1 - dist  # Similaridade: quanto menor a distância, maior a similaridade
        neighbor_items = interaction_matrix[neighbor_idx].nonzero()[1]
        for item in neighbor_items:
            if item in user_items:
                continue  # Ignorar itens já consumidos
            rec_scores[item] = rec_scores.get(item, 0) + similarity

    # Ordenar os itens por pontuação decrescente e retornar os top_n (convertendo os índices para IDs reais)
    ranked_items = sorted(rec_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_item_ids = [unique_items[idx] for idx, score in ranked_items[:top_n]]
    
    return recommended_item_ids

In [24]:
# Exemplo de recomendação CF para um usuário específico
user_id = "c1e8d644329a78ea1f994292db624c57980b2886cfbc2d49cc3ae12fb5a533af"
print("CF Recommendation para o usuário {}: {}".format(user_id, cf_recommend(user_id, top_n=5)))

CF Recommendation para o usuário c1e8d644329a78ea1f994292db624c57980b2886cfbc2d49cc3ae12fb5a533af: ['1f32787b-de2b-49be-8c20-ddaeae34cc22', 'bf257382-74fb-4392-ad6a-143240e39f81', 'c85cb6f6-60e1-4a94-8f8f-6b436bb1d77c', '96d328b9-5da7-4389-a9ac-765e98971ab7', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd']


In [25]:
user_idx = user_id_to_index[user_id]
user_interactions = interaction_matrix[user_idx].nonzero()[1]
print("Itens já consumidos pelo usuário:", [unique_items[i] for i in user_interactions])


Itens já consumidos pelo usuário: ['04756569-593e-4133-a95a-83d35d43dbbd', 'b03986b4-7887-45f2-8269-a66b9d73a361', '09d074f5-0cc9-4c31-9bb4-bc1c0d315675', '4154f103-9e39-49f5-8982-cdf13c576928', '1f2b9c2f-a2d2-4192-b009-09065da8ec23', '9958aafa-3a3a-416c-a2b9-5d96769bad83', '69990b31-a6f9-4db0-80c3-1d4733f9e619', 'd4303151-c778-461b-8207-2e443db4b255', '722623f2-349b-4e8b-a974-ad8995f747ff', '9930ad3b-a506-4129-8efd-20c573365d74', 'faac2a94-565a-4fad-9db6-3abbb6e39c8a', '6dfb5395-decf-4b1f-ae70-959bde7417e2', 'b4070b48-1df6-4e82-ab9d-f7547c8a6da7', 'f89f8706-ffd4-4d86-af65-b2d0236996f4', '197de720-5810-4e35-b50c-02a1692f0e5a', '39410577-0d94-4790-ac41-81362bc8c915', '5db25cd8-32e2-4502-9208-b62d0e13b015', '14a54c8d-3155-4045-83bc-ae01af95a034', 'bcb7755a-5046-4bd7-81cd-8ead61a27ba7', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', 'ab93d775-dc30-45a2-b885-9dfcc26fd154', '470ccd72-1478-4a82-9e75-ecb6f732772a', '73ed314a-a9f3-4b76-83f1-67063a4ca520', '4e9c2825-ff13-41ca-8e91-edd848060d19', '8c24

In [26]:
history_user_id = train_df[train_df['userId'] == user_id]['history'].values[0]
print("Histórico do usuário:", history_user_id)

Histórico do usuário: ['1f2b9c2f-a2d2-4192-b009-09065da8ec23', '04756569-593e-4133-a95a-83d35d43dbbd', 'b03986b4-7887-45f2-8269-a66b9d73a361', '9958aafa-3a3a-416c-a2b9-5d96769bad83', '69990b31-a6f9-4db0-80c3-1d4733f9e619', 'd4303151-c778-461b-8207-2e443db4b255', '722623f2-349b-4e8b-a974-ad8995f747ff', '9930ad3b-a506-4129-8efd-20c573365d74', 'faac2a94-565a-4fad-9db6-3abbb6e39c8a', '6dfb5395-decf-4b1f-ae70-959bde7417e2', '09d074f5-0cc9-4c31-9bb4-bc1c0d315675', 'b4070b48-1df6-4e82-ab9d-f7547c8a6da7', 'f89f8706-ffd4-4d86-af65-b2d0236996f4', '197de720-5810-4e35-b50c-02a1692f0e5a', '39410577-0d94-4790-ac41-81362bc8c915', '5db25cd8-32e2-4502-9208-b62d0e13b015', '14a54c8d-3155-4045-83bc-ae01af95a034', 'bcb7755a-5046-4bd7-81cd-8ead61a27ba7', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', 'ab93d775-dc30-45a2-b885-9dfcc26fd154', '470ccd72-1478-4a82-9e75-ecb6f732772a', '73ed314a-a9f3-4b76-83f1-67063a4ca520', '4154f103-9e39-49f5-8982-cdf13c576928', '4e9c2825-ff13-41ca-8e91-edd848060d19', '8c246d2b-81bd-4c

In [27]:
print(train_df.loc[train_df['userId'] == user_id, 'history'])

3    [1f2b9c2f-a2d2-4192-b009-09065da8ec23, 0475656...
Name: history, dtype: object


## Modelo Content-Based com TF-IDF

In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
# Obter a lista de stopwords em português
stopwords_pt = stopwords.words('portuguese')

In [31]:
# items_df['combined_text'] = items_df['title'].fillna('')

items_df['combined_text'] = (items_df['title'].fillna('') + " " +
                            #  items_df['body'].fillna('') + " " +
                             items_df['caption'].fillna(''))

# Inicializar o TfidfVectorizer com limite de features para economia de memória
tfidf = TfidfVectorizer(stop_words=stopwords_pt, max_features=1000)
tfidf_matrix = tfidf.fit_transform(items_df['combined_text'])

In [32]:
# Em vez de calcular a matriz de similaridade completa, usamos NearestNeighbors
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [33]:
def get_similar_items(item_id, top_n=10):
    """
    Dado um ID de item, retorna os top_n itens mais similares utilizando TF-IDF e NearestNeighbors.
    """
    if item_id not in items_df.index:
        return []
    
    # Obter o índice correspondente no tfidf_matrix
    item_idx = items_df.index.get_loc(item_id)
    query_vector = tfidf_matrix[item_idx]
    
    # Buscar os vizinhos mais próximos (n_neighbors inclui o próprio item, por isso solicitamos top_n+1)
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=top_n+1)
    
    # Ignorar o primeiro resultado (o próprio item) e mapear para os IDs dos itens
    similar_indices = indices.flatten()[1:]
    similar_item_ids = [items_df.index[i] for i in similar_indices]
    
    return similar_item_ids

In [ ]:
# Exemplo: buscar itens similares para um item específico
item_id_exemplo = items_df.index[50498]
print("Itens similares para o item {}: {}".format(item_id_exemplo, get_similar_items(item_id_exemplo, top_n=5)))

Itens similares para o item 50498: [164851, 194649, 230525, 172758, 224865]


In [42]:
items_df.iloc[50498]

page                          6b0450a3-bd80-4fe5-85f3-916458b65988
url              http://g1.globo.com/rj/rio-de-janeiro/noticia/...
issued                                   2021-10-02 13:28:50+00:00
modified                                 2021-10-02 15:37:46+00:00
title            Manifestantes fazem ato contra o presidente Bo...
body             Manifestantes fazem protesto contra o presiden...
caption          Além de pedir o impeachment do presidente, ato...
combined_text    Manifestantes fazem ato contra o presidente Bo...
Name: 50498, dtype: object

In [41]:
print(items_df.iloc[50498]['title'])
print(items_df.iloc[194649]['title'])
print(items_df.iloc[164851]['title'])
print(items_df.iloc[230525]['title'])
print(items_df.iloc[172758]['title'])
print(items_df.iloc[224865]['title'])

Manifestantes fazem ato contra o presidente Bolsonaro no Centro do Rio 
Manifestantes protestam no Centro do Rio contra o governo do presidente Jair Bolsonaro
Manifestantes fazem ato contra Bolsonaro no Centro do Rio
Manifestantes fazem ato contra Bolsonaro em São Luís
Manifestantes fazem ato em defesa da democracia e contra Bolsonaro em SP
Manifestantes se reúnem em ato pelo impeachment do presidente Jair Bolsonaro em Guararema


* 73027: [227748, 14414, 196714, 224865, 206553]
* 206553: [224865, 226625, 50498, 194649, 172758]
* >50498: [164851, 194649, 230525, 172758, 224865]
*   Manifestantes fazem ato contra o presidente Bolsonaro no Centro do Rio 
*   Manifestantes protestam no Centro do Rio contra o governo do presidente Jair Bolsonaro
*   Manifestantes fazem ato contra Bolsonaro no Centro do Rio
*   Manifestantes fazem ato contra Bolsonaro em São Luís
*   Manifestantes fazem ato em defesa da democracia e contra Bolsonaro em SP
*   Manifestantes se reúnem em ato pelo impeachment do presidente Jair Bolsonaro em Guararema

##  Abordagem Híbrida

In [43]:
def hybrid_recommend(user_id, top_n=10, weight_cf=0.5, weight_cb=0.5):
    """
    Combina as recomendações de Collaborative Filtering (CF) e Content-Based (CB) usando KNN.
    Para cada usuário, são obtidas as recomendações CF e, para o histórico do usuário,
    utiliza-se o KNN para buscar itens similares. Os scores de ambos os métodos são
    combinados de forma ponderada para gerar o ranking final.
    """
    # Recomendações baseadas em CF
    recs_cf = cf_recommend(user_id, top_n=top_n*2)
    
    # Obter o histórico do usuário a partir do dataset de treino
    user_history = []
    df_user = train_df[train_df['userId'] == user_id]
    for hist in df_user['history']:
        user_history.extend(hist)
    user_history = list(set(user_history))
    
    # Recomendações baseadas em conteúdo utilizando KNN
    # Para cada item do histórico, buscamos os itens mais similares
    score_cb = {}
    for item in user_history:
        similar_items = get_similar_items(item, top_n=10)  # Obtemos os 10 itens mais similares para cada item
        for rec in similar_items:
            # Ignoramos itens já consumidos
            if rec in user_history:
                continue
            # Acumula uma contagem que pode representar a força do sinal
            score_cb[rec] = score_cb.get(rec, 0) + 1

    # Ordena os itens recomendados pelo conteúdo com base no score acumulado
    recs_cb = [item for item, score in sorted(score_cb.items(), key=lambda x: x[1], reverse=True)][:top_n*2]
    
    # Combinação simples: somamos os pesos das recomendações CF e CB
    score_dict = {}
    for item in recs_cf:
        score_dict[item] = score_dict.get(item, 0) + weight_cf
    for item in recs_cb:
        score_dict[item] = score_dict.get(item, 0) + weight_cb
        
    # Ordena os itens combinados e retorna os top_n
    ranked_items = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return [item for item, score in ranked_items][:top_n]


In [44]:
# Exemplo de uso da recomendação híbrida:
print("Hybrid Recommendation para usuário {}: {}".format(user_id, hybrid_recommend(user_id=user_id, top_n=5)))

Hybrid Recommendation para usuário c1e8d644329a78ea1f994292db624c57980b2886cfbc2d49cc3ae12fb5a533af: ['1f32787b-de2b-49be-8c20-ddaeae34cc22', 'bf257382-74fb-4392-ad6a-143240e39f81', 'c85cb6f6-60e1-4a94-8f8f-6b436bb1d77c', '96d328b9-5da7-4389-a9ac-765e98971ab7', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd']


In [49]:
items_df[items_df["page"] == "1f32787b-de2b-49be-8c20-ddaeae34cc22"][["title","caption"]]

,title,caption
215724,Filha é presa por golpe estimado em R$ 725 mil...,"\nSegundo as investigações, a filha contratou ..."
